In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

folder_path = '/content/drive/MyDrive/시설/유지'
files = os.listdir(folder_path)
for f in files:
    print(f)

3월마감 조건추출 선택컬럼_2403.csv


In [ ]:
import pandas as pd

# 실제 출력된 정확한 파일명을 넣어주세요
file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 첫 5행 미리보기
df.head()

,관리본부명,관리지사명,고객번호,계약번호,서비스번호,서비스(대),서비스(중),서비스(소),상호,고객구분,...,계약종료일,영업자명,정지시작일자,정지희망종료일,계약최초서비스게시일,영업구역정보,제외사유,매출구분,실적채널,고알프
0,대구/경북본부,구미지사,30308344,50680565,60823184,기본서비스,텔레캅아이,ktt OCT 가드(DVR),세사리빙김천덕곡점(OCT),개인사업자,...,20260131,김찬운,NaN,NaN,2.016080e+13,M000306,NaN,NaN,SP,N
1,대구/경북본부,구미지사,30306886,50678019,60820188,기본서비스,GiGAeyes Basic,GiGAeyes Basic,CJ올리브영-김천혁신점,기타재단법인/협회,...,20260131,김찬운,NaN,NaN,2.016080e+13,M000306,NaN,AM(기업),AM,N
2,대구/경북본부,구미지사,30298490,50661989,60802044,기본서비스,텔레캅아이,ktt OCT 가드(DVR),경희한의원,개인사업자,...,20260131,김찬운,NaN,NaN,2.016070e+13,M000306,NaN,NaN,SP,N
3,대구/경북본부,구미지사,30609462,50646972,60784573,기본서비스,텔레캅아이,ktt OCT 가드(DVR),엠건축인테리어카페(OCT),개인사업자,...,20260131,김찬운,NaN,NaN,2.016080e+13,M000306,NaN,NaN,SP,N
4,대구/경북본부,구미지사,30514485,50895274,61073397,기본서비스,텔레캅아이,ktt GiGAeyes view+(DVR),크린업24시김천시청점,개인사업자,...,20251001,김찬운,NaN,NaN,2.016070e+13,M000306,NaN,NaN,SP,N


In [ ]:
import pandas as pd

# ✅ 파일 경로
file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv'

# ✅ CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8-sig')

# ✅ 필수 컬럼만 추출
df_subset = df[['관리본부명', '계약번호']].dropna()

# ✅ 계약번호 기준으로 중복 제거
df_unique = df_subset.drop_duplicates(subset='계약번호')

# ✅ 관리본부명 기준으로 계약 건수 집계
summary = df_unique.groupby('관리본부명').count().reset_index()
summary.columns = ['관리본부명', '계약건수']

# ✅ 결과 출력
from IPython.display import display
display(summary)

,관리본부명,계약건수
0,강남/서부본부,62866
1,강북/강원본부,65391
2,강원본부,24651
3,대구/경북본부,46347
4,부산/경남본부,62203
5,서부본부,57086
6,전남/전북본부,66406
7,충남/충북본부,50018


In [ ]:
# ✅ 기본 모듈
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar

# ✅ 파일 경로 설정
file_path = '/content/drive/MyDrive/시설/유지/C150_G0000_00012.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/2025/3월/3월마감 조건추출 선택컬럼_2403.csv'
output_excel_path = '/content/drive/MyDrive/시설/유지/2025년0421_전사 정지조건리스트_완성.xlsx'
output_cs_path = '/content/drive/MyDrive/시설/유지/2025년0421_전사 정지조건리스트_완성.csv'

# ✅ 인코딩 자동 감지
def try_read_csv(path):
    encodings = ['utf-8-sig', 'cp949', 'euc-kr']
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except:
            continue
    raise ValueError(f"❌ 파일을 열 수 없습니다: {path}")

# ✅ 파일 읽기
df = try_read_csv(file_path)
additional_df = try_read_csv(additional_file_path)

# ✅ 컬럼 선택 및 전처리
columns_to_extract = [
    '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
    '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', '계약시작일', '계약종료일',
    '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
]
df = df[[col for col in columns_to_extract if col in df.columns]]
df = df.rename(columns={'KTT월정료(조정)': 'KTT월정료'})
df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

# ✅ 병합할 추가 컬럼
additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
additional_df = additional_df[['계약번호'] + additional_columns].drop_duplicates(subset='계약번호')

# ✅ 병합
for col in additional_columns:
    df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

# ✅ 제외사유 제거
df = df[df['제외사유'].astype(str).str.strip() == '']

# ✅ 날짜 정제 함수
def convert_date(x):
    x = str(x).strip()
    if len(x) >= 8 and x.isdigit():
        return f"{x[:4]}-{x[4:6]}-{x[6:8]}"
    return None

for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
    df[col] = df[col].apply(convert_date)

# ✅ 종료일 보정
today = datetime.today()
last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
last_day_str = last_day.strftime("%Y-%m-%d")

df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x or x == '9999-12-31' else x)

# ✅ 정지일수 계산
def calc_days(s, e):
    try:
        s = datetime.strptime(s, "%Y-%m-%d")
        e = datetime.strptime(e, "%Y-%m-%d")
        return max((e - s).days + 1, 0)
    except:
        return None

df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

# ✅ 정지일수 구간 분류
def categorize_freeze(days):
    if pd.isna(days): return None
    elif days <= 89: return '89일이하'
    elif days <= 119: return '90~119일'
    elif days <= 149: return '120~149일'
    else: return '150일이상'

df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze)

# ✅ 종료희망일 초과 여부
df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
    lambda d: '초과' if pd.to_datetime(d, errors='coerce') <= last_day else '미초과'
)

# ✅ 월정료 구간
def fee_range(val):
    try:
        v = float(str(val).replace(',', ''))
        if v <= 50000: return '5만원 이하'
        elif v <= 70000: return '5만원 초과 ~ 7만원 이하'
        elif v <= 100000: return '7만원 초과 ~ 10만원 이하'
        elif v <= 200000: return '10만원 초과 ~ 20만원 이하'
        else: return '20만원 초과'
    except:
        return None

df['월정료 구간'] = df['KTT월정료'].apply(fee_range)

# ✅ 컬럼 정렬
new_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
df = df[[col for col in new_columns if col in df.columns]]

# ✅ 엑셀로 저장
df.to_excel(output_excel_path, index=False, engine='openpyxl')

# ✅ 노란색 음영 처리
wb = load_workbook(output_excel_path)
ws = wb.active
yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
for col in range(1, len(additional_columns) + 5):
    for row in range(2, ws.max_row + 1):
        ws.cell(row=row, column=col).fill = yellow_fill
wb.save(output_excel_path)

# ✅ CSV 저장
df.to_csv(output_cs_path, index=False, encoding='cp949')

print(f"✅ 저장 완료! 최종 행 수: {df.shape[0]}")

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar

# ✅ 인코딩 자동 탐색 함수
def try_read_csv(path):
    encodings = ['utf-8', 'cp949', 'euc-kr', 'ISO-8859-1']
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            continue
    raise ValueError(f"❌ 파일을 읽을 수 없습니다: {path}")

# ✅ 파일 경로
file_path = r'D:\시설\정지\일일정지\20250421\C150_G0000_00012.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250421\2025년0421_전사 정지조건리스트_완성.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250421\2025년0421_전사 정지조건리스트_완성.cs'

# ✅ 데이터 불러오기
df = try_read_csv(file_path)
additional_df = try_read_csv(additional_file_path)

# ✅ 컬럼 지정
columns_to_extract = [
    '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)', '상호',
    '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', '계약시작일', '계약종료일',
    '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
]
df = df[[col for col in columns_to_extract if col in df.columns]]
df = df.rename(columns={'KTT월정료(조정)': 'KTT월정료'})

# ✅ 필터링
df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

# ✅ 병합 대상 컬럼
additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
additional_df = additional_df[['계약번호'] + additional_columns].drop_duplicates(subset='계약번호')

# ✅ 병합
for col in additional_columns:
    df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

# ✅ 제외사유 제거
df = df[df['제외사유'].astype(str).str.strip() == '']

# ✅ 날짜 정제 함수
def convert_date(x):
    x = str(x).strip()
    if len(x) >= 8 and x.isdigit():
        return f"{x[:4]}-{x[4:6]}-{x[6:8]}"
    return None

for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
    df[col] = df[col].apply(convert_date)

# ✅ 정지희망종료일이 비어있거나 이상치인 경우 당월 말일 적용
today = datetime.today()
last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
last_day_str = last_day.strftime("%Y-%m-%d")

df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x or x == '9999-12-31' else x)

# ✅ 정지일수 계산
def calc_days(s, e):
    try:
        s = datetime.strptime(s, "%Y-%m-%d")
        e = datetime.strptime(e, "%Y-%m-%d")
        return max((e - s).days + 1, 0)
    except:
        return None

df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

# ✅ 정지일수 구간
def categorize_freeze(days):
    if pd.isna(days):
        return None
    elif days <= 89:
        return '89일이하'
    elif days <= 119:
        return '90~119일'
    elif days <= 149:
        return '120~149일'
    else:
        return '150일이상'

df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze)

# ✅ 종료희망일 초과 여부
def check_exceeding(end_date_str):
    try:
        end = datetime.strptime(end_date_str, '%Y-%m-%d')
        return '초과' if end <= last_day else '미초과'
    except:
        return None

df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(check_exceeding)

# ✅ 월정료 구간
def fee_range(val):
    try:
        v = float(str(val).replace(',', ''))
        if v <= 50000:
            return '5만원 이하'
        elif v <= 70000:
            return '5만원 초과 ~ 7만원 이하'
        elif v <= 100000:
            return '7만원 초과 ~ 10만원 이하'
        elif v <= 200000:
            return '10만원 초과 ~ 20만원 이하'
        else:
            return '20만원 초과'
    except:
        return None

df['월정료 구간'] = df['KTT월정료'].apply(fee_range)

# ✅ 컬럼 정렬
new_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
df = df[[col for col in new_columns if col in df.columns]]

# ✅ 엑셀 저장
df.to_excel(output_excel_path, index=False, engine='openpyxl')

# ✅ 노란색 음영 처리
wb = load_workbook(output_excel_path)
ws = wb.active
yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
for col in range(1, len(additional_columns) + 5):
    for row in range(2, ws.max_row + 1):
        ws.cell(row=row, column=col).fill = yellow_fill
wb.save(output_excel_path)

# ✅ CSV 저장
df.to_csv(output_cs_path, index=False, encoding='cp949')

print(f"✅ 저장 완료! 최종 행 수: {df.shape[0]}")

ValueError: ❌ 파일을 읽을 수 없습니다: D:\시설\정지\일일정지\20250421\C150_G0000_00012.csv

In [ ]:
# ✅ 패키지 재설치
!pip install geopandas gspread gspread_dataframe folium oauth2client

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,관리본부명,관리지사명,고객번호,계약번호,서비스번호,서비스(대),서비스(중),서비스(소),상호,고객구분,...,계약종료일,영업자명,정지시작일자,정지희망종료일,계약최초서비스게시일,영업구역정보,제외사유,매출구분,실적채널,고알프
0,대구/경북본부,구미지사,30308344,50680565,60823184,기본서비스,텔레캅아이,ktt OCT 가드(DVR),세사리빙김천덕곡점(OCT),개인사업자,...,20260131,김찬운,NaN,NaN,2.016080e+13,M000306,NaN,NaN,SP,N
1,대구/경북본부,구미지사,30306886,50678019,60820188,기본서비스,GiGAeyes Basic,GiGAeyes Basic,CJ올리브영-김천혁신점,기타재단법인/협회,...,20260131,김찬운,NaN,NaN,2.016080e+13,M000306,NaN,AM(기업),AM,N
2,대구/경북본부,구미지사,30298490,50661989,60802044,기본서비스,텔레캅아이,ktt OCT 가드(DVR),경희한의원,개인사업자,...,20260131,김찬운,NaN,NaN,2.016070e+13,M000306,NaN,NaN,SP,N
3,대구/경북본부,구미지사,30609462,50646972,60784573,기본서비스,텔레캅아이,ktt OCT 가드(DVR),엠건축인테리어카페(OCT),개인사업자,...,20260131,김찬운,NaN,NaN,2.016080e+13,M000306,NaN,NaN,SP,N
4,대구/경북본부,구미지사,30514485,50895274,61073397,기본서비스,텔레캅아이,ktt GiGAeyes view+(DVR),크린업24시김천시청점,개인사업자,...,20251001,김찬운,NaN,NaN,2.016070e+13,M000306,NaN,NaN,SP,N


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
streamlit run streamlit_app.py

SyntaxError: invalid syntax (<ipython-input-7-5124338ac38b>, line 1)

In [ ]:
# 1. 필수 패키지 설치
!pip install streamlit pyngrok --quiet

# 2. ngrok 토큰 등록
from pyngrok import conf, ngrok

conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"

# 3. Streamlit 앱 코드 작성
code = '''
import streamlit as st
import pandas as pd

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 본부명 통합
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

df = df[['관리본부명', '관리지사명', '계약번호', '상호']].dropna()
df = df.drop_duplicates(subset='계약번호')

summary = df.groupby(['관리본부명', '관리지사명']).size().reset_index(name='계약건수')

st.set_page_config(layout="wide")
st.title("계약 현황 대시보드")
search = st.text_input("계약번호 또는 상호 검색")

if search:
    result = df[df['계약번호'].astype(str).str.contains(search) | df['상호'].str.contains(search)]
    st.dataframe(result)
else:
    st.dataframe(summary)
'''

# 4. 파일로 저장
with open('app.py', 'w', encoding='utf-8') as f:
    f.write(code)

# 5. Streamlit 앱 실행
!streamlit run --server.port 8501 app.py &>/dev/null &

# 6. ngrok을 통해 외부 접속 URL 생성
public_url = ngrok.connect(8501)
print(f"✅ 앱이 실행 중입니다. 아래 링크를 클릭해보세요:\n{public_url}")

✅ 앱이 실행 중입니다. 아래 링크를 클릭해보세요:
NgrokTunnel: "https://4d9a-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install plotly --quiet

In [ ]:
import streamlit as st
import pandas as pd

# 파일 경로
file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 본부명 통합
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

# 필수 컬럼 정리
df = df[['관리본부명', '관리지사명', '영업자명', '계약번호', '상호', 'KTT월정료(조정)']].dropna()
df = df.drop_duplicates(subset='계약번호')
df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'], errors='coerce').fillna(0)

# Streamlit 설정
st.set_page_config(layout="wide")
st.title("계약 현황 집계 대시보드")

# 검색
search = st.text_input("계약번호 또는 상호 검색")

if search:
    result = df[df['계약번호'].astype(str).str.contains(search) | df['상호'].str.contains(search)]
    st.dataframe(result)
else:
    # 1. 본부별 소계
    st.subheader("▶️ 본부별 계약 건수 및 월정료 합계")
    total_summary = df.groupby('관리본부명').agg(계약건수=('계약번호', 'count'), 월정료합계=('KTT월정료(조정)', 'sum')).reset_index()
    st.dataframe(total_summary)

    # 2. 본부 + 지사별 소계
    st.subheader("▶️ 본부 + 지사별 소계")
    region_summary = df.groupby(['관리본부명', '관리지사명']).agg(계약건수=('계약번호', 'count'), 월정료합계=('KTT월정료(조정)', 'sum')).reset_index()
    st.dataframe(region_summary)

    # 3. 영업자별 현황
    st.subheader("▶️ 영업자별 계약 현황")
    sales_summary = df.groupby('영업자명').agg(계약건수=('계약번호', 'count'), 월정료합계=('KTT월정료(조정)', 'sum')).reset_index().sort_values(by='월정료합계', ascending=False)
    st.dataframe(sales_summary)

    # 4. 전체 합계
    st.subheader("▶️ 전체 합계")
    total_contracts = df['계약번호'].nunique()
    total_fee = df['KTT월정료(조정)'].sum()
    st.metric("총 계약 건수", f"{total_contracts:,} 건")
    st.metric("총 KTT 월정료(조정)", f"{total_fee:,.0f} 원")

2025-04-21 18:35:48.322 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.331 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.331 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:35:48.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# ✅ 페이지 설정
st.set_page_config(layout="wide")
st.title("▶️ 관리본부별 계약현황 및 KTT월정료(조정) 시각화")

# ✅ CSV 파일 로딩
file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# ✅ 본부명 정리
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

# ✅ 컬럼명 변경 및 안전 복사
df = df.rename(columns={'KTT월정료': 'KTT월정료(조정)'})
df = df[['관리본부명', '계약번호', 'KTT월정료(조정)']].copy()
df.dropna(inplace=True)
df.drop_duplicates(subset='계약번호', inplace=True)
df.loc[:, 'KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'], errors='coerce').fillna(0)

# ✅ 집계 처리
summary = df.groupby('관리본부명').agg(
    계약건수=('계약번호', 'count'),
    월정료합계=('KTT월정료(조정)', 'sum')
).reset_index()

summary['월정료_백만'] = summary['월정료합계'] / 1_000_000

# ✅ 시각화
fig = go.Figure()

# 계약건수
fig.add_trace(go.Bar(
    x=summary['관리본부명'],
    y=summary['계약건수'],
    name='계약건수',
    text=summary['계약건수'],
    textposition='outside',
    yaxis='y1'
))

# 월정료
fig.add_trace(go.Bar(
    x=summary['관리본부명'],
    y=summary['월정료_백만'],
    name='KTT월정료(조정) (백만)',
    text=summary['월정료_백만'].round(1),
    textposition='outside',
    yaxis='y2'
))

# 그래프 설정
fig.update_layout(
    title='관리본부별 계약건수 및 KTT월정료(조정)',
    xaxis=dict(title='관리본부명'),
    yaxis=dict(title='계약건수', side='left'),
    yaxis2=dict(title='KTT월정료(백만)', overlaying='y', side='right', showgrid=False),
    barmode='group',
    height=500
)

# ✅ Streamlit 출력
st.plotly_chart(fig, use_container_width=True)
st.caption("✅ 그래프가 이 아래에 보이지 않으면 summary 데이터가 비어있거나 그래프 생성 코드가 조건문 밖에 없을 수 있습니다.")

2025-04-21 18:28:36.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:36.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:36.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:41.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:41.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:41.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:41.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:28:41.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [ ]:
!streamlit run app.py &>/content/log.txt &
from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
public_url = ngrok.connect(8501)
print(public_url)

NgrokTunnel: "https://74c4-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
if not summary.empty:
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=summary['관리본부명'],
        y=summary['계약건수'],
        name='계약건수',
        text=summary['계약건수'],
        textposition='outside',
        yaxis='y1'
    ))

    fig.add_trace(go.Bar(
        x=summary['관리본부명'],
        y=summary['월정료_백만'],
        name='KTT월정료(조정) (백만)',
        text=summary['월정료_백만'].round(1),
        textposition='outside',
        yaxis='y2'
    ))

    fig.update_layout(
        title='관리본부별 계약건수 및 KTT월정료(조정)',
        xaxis=dict(title='관리본부명'),
        yaxis=dict(title='계약건수', side='left'),
        yaxis2=dict(title='KTT월정료(백만)', overlaying='y', side='right', showgrid=False),
        barmode='group',
        height=500
    )

    st.subheader("▶️ 관리본부별 계약건수 & KTT월정료(조정) 시각화")
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("⚠️ 시각화할 summary 데이터가 없습니다.")

2025-04-21 18:29:37.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:29:37.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:29:37.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:29:37.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:29:37.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:29:37.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
!streamlit run app.py &>/content/log.txt &
from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
public_url = ngrok.connect(8501)
print(public_url)

NgrokTunnel: "https://23db-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import pandas as pd
import streamlit as st
import plotly.graph_objects as go

df = pd.DataFrame({
    '관리본부명': ['A', 'B', 'C'],
    '계약건수': [100, 200, 150],
    '월정료_백만': [50, 75, 60]
})

fig = go.Figure()
fig.add_trace(go.Bar(x=df['관리본부명'], y=df['계약건수'], name='계약건수', text=df['계약건수'], textposition='outside'))
fig.add_trace(go.Bar(x=df['관리본부명'], y=df['월정료_백만'], name='KTT월정료', text=df['월정료_백만'], textposition='outside'))

fig.update_layout(
    title='테스트 그래프',
    barmode='group'
)

st.plotly_chart(fig, use_container_width=True)

2025-04-21 18:36:20.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:36:20.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:36:20.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:36:20.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
!streamlit run app.py &>/content/log.txt &
from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
public_url = ngrok.connect(8501)
print(public_url)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2w3AZAGXwsJqEx8co9zb4okG6ed, tn_2w3AxP3J4zVizfpiSusCrCZ12jd, tn_2w3B5cvrCMWdcmaKSpiEatdFPoM\n\r\n\r\nERR_NGROK_324\r\n"}}


In [ ]:
# ✅ 1. 필요한 패키지 설치
!pip install streamlit pyngrok plotly --quiet

# ✅ 2. ngrok 토큰 등록 (★ 반드시 본인 토큰 입력)
from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"

# ✅ 3. 기존 ngrok 터널 강제 종료
ngrok.kill()

# ✅ 4. Streamlit 앱 코드 저장
app_code = '''
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# 페이지 설정
st.set_page_config(layout="wide")
st.title("▶️ 관리본부별 계약현황 및 KTT월정료(조정) 시각화")

# 파일 경로
file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 본부명 통합
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

# 컬럼명 변경 및 정리
df = df.rename(columns={'KTT월정료': 'KTT월정료(조정)'})
df = df[['관리본부명', '계약번호', 'KTT월정료(조정)']].copy()
df.dropna(inplace=True)
df.drop_duplicates(subset='계약번호', inplace=True)
df.loc[:, 'KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'], errors='coerce').fillna(0)

# 집계
summary = df.groupby('관리본부명').agg(
    계약건수=('계약번호', 'count'),
    월정료합계=('KTT월정료(조정)', 'sum')
).reset_index()
summary['월정료_백만'] = summary['월정료합계'] / 1_000_000

# 그래프
if not summary.empty:
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=summary['관리본부명'],
        y=summary['계약건수'],
        name='계약건수',
        text=summary['계약건수'],
        textposition='outside',
        yaxis='y1'
    ))

    fig.add_trace(go.Bar(
        x=summary['관리본부명'],
        y=summary['월정료_백만'],
        name='KTT월정료(조정) (백만)',
        text=summary['월정료_백만'].round(1),
        textposition='outside',
        yaxis='y2'
    ))

    fig.update_layout(
        title='관리본부별 계약건수 및 KTT월정료(조정)',
        xaxis=dict(title='관리본부명'),
        yaxis=dict(title='계약건수', side='left'),
        yaxis2=dict(title='KTT월정료(백만)', overlaying='y', side='right', showgrid=False),
        barmode='group',
        height=500
    )

    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("⚠️ 시각화할 summary 데이터가 없습니다.")
'''

# 저장
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

# ✅ 5. Streamlit 앱 실행
!streamlit run app.py &>/content/log.txt &

# ✅ 6. ngrok으로 외부 접속 URL 생성
public_url = ngrok.connect(8501)
print(f"✅ 앱 실행 중: {public_url}")

✅ 앱 실행 중: NgrokTunnel: "https://ef64-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# 페이지 설정
st.set_page_config(layout="wide")
st.title("▶️ 계약 현황 대시보드")

# CSV 파일 경로
file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 본부명 정리
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

# KTT월정료 → KTT월정료(조정)으로 변경
if 'KTT월정료' in df.columns:
    df = df.rename(columns={'KTT월정료': 'KTT월정료(조정)'})
    df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'], errors='coerce').fillna(0)
else:
    df['KTT월정료(조정)'] = 0

# ✅ 검색
search = st.text_input("계약번호 또는 상호 검색")

if search:
    filtered = df[
        df['계약번호'].astype(str).str.contains(search) |
        df['상호'].astype(str).str.contains(search)
    ]
    st.subheader("▶️ 검색 결과")
    st.dataframe(filtered)
else:
    st.subheader("▶️ 전체 계약 현황")
    st.dataframe(df)

# ✅ 요약 통계 (전체 기준)
grouped = df.groupby('관리본부명').agg(
    계약건수=('계약번호', 'nunique'),
    월정료합계=('KTT월정료(조정)', 'sum')
).reset_index()
grouped['월정료_백만'] = grouped['월정료합계'] / 1_000_000

# ✅ 시각화
if not grouped.empty:
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=grouped['관리본부명'],
        y=grouped['계약건수'],
        name='계약건수',
        text=grouped['계약건수'],
        textposition='outside',
        yaxis='y1'
    ))

    fig.add_trace(go.Bar(
        x=grouped['관리본부명'],
        y=grouped['월정료_백만'],
        name='KTT월정료(조정) (백만)',
        text=grouped['월정료_백만'].round(1),
        textposition='outside',
        yaxis='y2'
    ))

    fig.update_layout(
        title='관리본부별 계약건수 및 KTT월정료(조정)',
        xaxis=dict(title='관리본부명'),
        yaxis=dict(title='계약건수', side='left'),
        yaxis2=dict(title='KTT월정료(백만)', overlaying='y', side='right', showgrid=False),
        barmode='group',
        height=550
    )

    st.subheader("▶️ 관리본부별 계약건수 & KTT월정료(조정) 시각화")
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("❗ 시각화할 데이터가 없습니다.")

2025-04-21 18:41:42.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:42.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:42.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:50.070 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:50.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:50.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:50.077 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:41:50.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!streamlit run app.py &>/content/log.txt &
from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
public_url = ngrok.connect(8501)
print(public_url)

NgrokTunnel: "https://45ea-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# ✅ 1. 설치 및 초기화
!pip install streamlit pyngrok plotly --quiet

from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()

# ✅ 2. 앱 코드 저장
app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(layout="wide")
st.title("▶️ 계약현황 전문가 대시보드")

# ✅ 데이터 불러오기
file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# ✅ 컬럼 정리
df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})

# 금액 컬럼 자동 감지 및 변환
fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 주소 컬럼에서 시군구, 읍면동 추출
if '설치주소' in df.columns:
    df['시군구'] = df['설치주소'].str.extract(r'([가-힣]+시|[가-힣]+군|[가-힣]+구)')
    df['읍면동'] = df['설치주소'].str.extract(r'([가-힣]+읍|[가-힣]+면|[가-힣]+동)')

# ✅ 필터 UI
st.sidebar.header("필터")
selected_head = st.sidebar.selectbox("관리본부 선택", ['전체'] + sorted(df['관리본부명'].dropna().unique()))
selected_fee_col = st.sidebar.selectbox("금액 기준 컬럼", fee_cols)
min_fee = st.sidebar.number_input("최소 금액 (예: 300000)", value=300000)
selected_city = st.sidebar.selectbox("시군구", ['전체'] + sorted(df['시군구'].dropna().unique()))
selected_town = st.sidebar.selectbox("읍면동", ['전체'] + sorted(df['읍면동'].dropna().unique()))

# ✅ 필터링
filtered_df = df.copy()
if selected_head != '전체':
    filtered_df = filtered_df[filtered_df['관리본부명'] == selected_head]
if selected_city != '전체':
    filtered_df = filtered_df[filtered_df['시군구'] == selected_city]
if selected_town != '전체':
    filtered_df = filtered_df[filtered_df['읍면동'] == selected_town]
filtered_df = filtered_df[filtered_df[selected_fee_col] >= min_fee]

# ✅ 검색
search = st.text_input("계약번호 또는 상호 검색")
if search:
    filtered_df = filtered_df[
        filtered_df['계약번호'].astype(str).str.contains(search) |
        filtered_df['상호'].astype(str).str.contains(search)
    ]

# ✅ 전체 테이블 출력
st.subheader("▶️ 전체 결과 (필터 적용됨)")
st.dataframe(filtered_df)

# ✅ 지사 요약
if selected_head != '전체':
    st.subheader(f"▶️ {selected_head} 지사별 요약")
    summary = (
        filtered_df.groupby('관리지사명')
        .agg(계약건수=('계약번호', 'nunique'), 금액합계=(selected_fee_col, 'sum'))
        .reset_index()
    )
    summary['금액(백만)'] = (summary['금액합계'] / 1_000_000).round(2)
    st.dataframe(summary)

    fig = px.bar(summary, x='관리지사명', y='금액(백만)', text='계약건수', title=f"{selected_head} 지사별 금액")
    st.plotly_chart(fig, use_container_width=True)

# ✅ 전문가 기법: Top 10 영업자
if '영업자명' in filtered_df.columns:
    st.subheader("▶️ Top 10 영업자별 실적")
    top_sales = (
        filtered_df.groupby('영업자명')
        .agg(계약건수=('계약번호', 'nunique'), 금액합계=(selected_fee_col, 'sum'))
        .sort_values(by='금액합계', ascending=False)
        .head(10)
        .reset_index()
    )
    top_sales['금액(백만)'] = (top_sales['금액합계'] / 1_000_000).round(2)
    st.dataframe(top_sales)

    fig2 = px.bar(top_sales, x='영업자명', y='금액(백만)', text='계약건수', title='Top 10 영업자별 금액 실적')
    st.plotly_chart(fig2, use_container_width=True)
'''

# 저장
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

# ✅ 3. 앱 실행
!streamlit run app.py &>/content/log.txt &

# ✅ 4. ngrok으로 외부 URL 생성
public_url = ngrok.connect(8501)
print(f"✅ 앱 실행 중입니다. 접속 링크:\n{public_url}")

✅ 앱 실행 중입니다. 접속 링크:
NgrokTunnel: "https://454f-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
st.plotly_chart(fig, use_container_width=True)
st.write(fig)  # 혹시 Streamlit 버전이 plotly 지원이 약할 경우 대비

2025-04-21 18:49:54.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.084 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:49:54.098 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
# ✅ 1. 패키지 설치
!pip install streamlit pyngrok plotly wordcloud matplotlib --quiet

from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()

# ✅ 2. Streamlit 앱 코드 저장
app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 데이터 전문가 분석 대시보드")

# ✅ 데이터 불러오기
file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# ✅ 기본 정제
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# ✅ 1. 지사별 계약건수 및 금액
st.subheader("1️⃣ 지사별 계약건수 및 금액")
if '관리지사명' in df.columns:
    summary = df.groupby('관리지사명').agg(
        계약건수=('계약번호', 'nunique'),
        총금액=(fee_cols[0], 'sum')
    ).reset_index()
    summary['총금액(백만)'] = (summary['총금액'] / 1_000_000).round(2)
    st.dataframe(summary)
    fig1 = px.bar(summary, x='관리지사명', y='총금액(백만)', text='계약건수', title='지사별 금액 및 계약건수')
    st.plotly_chart(fig1, use_container_width=True)

# ✅ 2. 관리본부별 계약건수 및 총금액
st.subheader("2️⃣ 관리본부별 요약")
if '관리본부명' in df.columns:
    hq = df.groupby('관리본부명').agg(
        계약건수=('계약번호', 'nunique'),
        총금액=(fee_cols[0], 'sum')
    ).reset_index()
    hq['총금액(백만)'] = (hq['총금액'] / 1_000_000).round(2)
    st.dataframe(hq)
    fig2 = px.bar(hq, x='관리본부명', y='총금액(백만)', text='계약건수', title='본부별 금액 및 계약건수')
    st.plotly_chart(fig2, use_container_width=True)

# ✅ 3. 영업자별 Top 10 분석
st.subheader("3️⃣ Top 10 영업자 (금액 기준)")
if '영업자명' in df.columns:
    sales = df.groupby('영업자명').agg(
        계약건수=('계약번호', 'nunique'),
        총금액=(fee_cols[0], 'sum')
    ).sort_values(by='총금액', ascending=False).head(10).reset_index()
    sales['총금액(백만)'] = (sales['총금액'] / 1_000_000).round(2)
    st.dataframe(sales)
    fig3 = px.bar(sales, x='영업자명', y='총금액(백만)', text='계약건수', title='Top 10 영업자 실적')
    st.plotly_chart(fig3, use_container_width=True)

# ✅ 4. 상호 키워드 워드클라우드
st.subheader("4️⃣ 상호 키워드 분석")
if '상호' in df.columns:
    text = ' '.join(df['상호'].dropna().astype(str).tolist())
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig4, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig4)
'''

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

# ✅ 3. Streamlit 실행
!streamlit run app.py &>/content/log.txt &

# ✅ 4. ngrok URL 생성
public_url = ngrok.connect(8501)
print(f"✅ 앱 실행 중입니다. 접속 링크:\n{public_url}")

✅ 앱 실행 중입니다. 접속 링크:
NgrokTunnel: "https://1d59-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# ✅ 기존 설치 및 ngrok 세션 종료 유지
!pip install streamlit pyngrok plotly wordcloud matplotlib --quiet
from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()

# ✅ 시각화 포함된 완성 앱 저장
app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 전체 데이터 기반 전문가 대시보드")

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})
fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

st.markdown("### 1️⃣ 지사별 계약건수 및 금액 요약")
try:
    g1 = df.groupby('관리지사명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g1['총금액(백만)'] = g1['총금액'] / 1_000_000
    st.dataframe(g1)
    st.plotly_chart(px.bar(g1, x='관리지사명', y='총금액(백만)', text='계약건수', title="지사별 요약"), use_container_width=True)
except Exception as e:
    st.warning(f"지사 분석 불가: {e}")

st.markdown("### 2️⃣ 본부별 계약건수 및 금액 요약")
try:
    g2 = df.groupby('관리본부명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g2['총금액(백만)'] = g2['총금액'] / 1_000_000
    st.dataframe(g2)
    st.plotly_chart(px.bar(g2, x='관리본부명', y='총금액(백만)', text='계약건수', title="본부별 요약"), use_container_width=True)
except Exception as e:
    st.warning(f"본부 분석 불가: {e}")

st.markdown("### 3️⃣ 영업자 Top 10")
try:
    top_sales = df.groupby('영업자명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum'))
    top_sales = top_sales.sort_values(by='총금액', ascending=False).head(10).reset_index()
    top_sales['총금액(백만)'] = top_sales['총금액'] / 1_000_000
    st.dataframe(top_sales)
    st.plotly_chart(px.bar(top_sales, x='영업자명', y='총금액(백만)', text='계약건수', title="Top 10 영업자"), use_container_width=True)
except Exception as e:
    st.warning(f"영업자 분석 불가: {e}")

st.markdown("### 4️⃣ 상호 키워드 워드클라우드")
try:
    text = ' '.join(df['상호'].dropna().astype(str))
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)
except Exception as e:
    st.warning(f"워드클라우드 생성 실패: {e}")
'''

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

!streamlit run app.py &>/content/log.txt &
public_url = ngrok.connect(8501)
print(f"✅ 앱 실행 링크:\n{public_url}")

✅ 앱 실행 링크:
NgrokTunnel: "https://f8eb-34-139-51-30.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# ✅ 1. 필수 패키지 설치 및 ngrok 초기화
!pip install streamlit pyngrok plotly wordcloud matplotlib --quiet

from pyngrok import conf, ngrok
conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()

# ✅ 2. Streamlit 앱 코드 저장
app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 데이터 전문가 분석 대시보드 (전체 조건 적용 안함)")

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 본부 통합
df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})

# 금액 컬럼 자동 인식
fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# ✅ 지사별 요약
st.subheader("1️⃣ 지사별 계약건수 및 총금액")
if '관리지사명' in df.columns:
    g1 = df.groupby('관리지사명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    st.dataframe(g1)
    st.plotly_chart(px.bar(g1, x='관리지사명', y='총금액', text='계약건수', title="지사별 요약"), use_container_width=True)

# ✅ 본부별 요약
st.subheader("2️⃣ 본부별 계약건수 및 총금액")
if '관리본부명' in df.columns:
    g2 = df.groupby('관리본부명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    st.dataframe(g2)
    st.plotly_chart(px.bar(g2, x='관리본부명', y='총금액', text='계약건수', title="본부별 요약"), use_container_width=True)

# ✅ 영업자 Top 10
st.subheader("3️⃣ 영업자별 Top 10 실적")
if '영업자명' in df.columns:
    g3 = df.groupby('영업자명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g3 = g3.sort_values(by='총금액', ascending=False).head(10)
    st.dataframe(g3)
    st.plotly_chart(px.bar(g3, x='영업자명', y='총금액', text='계약건수', title="Top 10 영업자"), use_container_width=True)

# ✅ 상호 워드클라우드
st.subheader("4️⃣ 상호 키워드 워드클라우드")
if '상호' in df.columns:
    text = ' '.join(df['상호'].dropna().astype(str))
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)
'''

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

# ✅ 3. 실행 및 링크 출력
!streamlit run app.py &>/content/log.txt &
public_url = ngrok.connect(8501)
print(f"✅ 앱 실행 중입니다. 접속 링크:\n{public_url}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 145.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00
✅ 앱 실행 중입니다. 접속 링크:
NgrokTunnel: "https://1e71-34-125-201-77.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install streamlit pyngrok plotly wordcloud matplotlib --quiet

In [ ]:
app_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 데이터 전문가 분석 대시보드")

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})
df['관리지사명'] = df['관리지사명'].replace({'수성지사': '동대구지사', '춘천고객지원팀': '원주지사'})

fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

with st.sidebar:
    st.header("🔎 조건 검색")
    col1, col2 = st.columns(2)
    with col1:
        본부선택 = st.multiselect("관리본부명", options=sorted(df['관리본부명'].dropna().unique()), default=[])
    with col2:
        지사선택 = st.multiselect("관리지사명", options=sorted(df['관리지사명'].dropna().unique()), default=[])

    금액조건 = st.number_input("월정료 이상 금액", min_value=0, value=0)
    키워드 = st.text_input("계약번호 or 상호 검색", "")

filtered = df.copy()
if 본부선택:
    filtered = filtered[filtered['관리본부명'].isin(본부선택)]
if 지사선택:
    filtered = filtered[filtered['관리지사명'].isin(지사선택)]
if 금액조건 > 0 and fee_cols:
    filtered = filtered[filtered[fee_cols[0]] >= 금액조건]
if 키워드.strip():
    filtered = filtered[
        filtered['계약번호'].astype(str).str.contains(키워드, case=False) |
        filtered['상호'].astype(str).str.contains(키워드, case=False)
    ]

st.markdown(f"**데이터 건수: {len(filtered):,}건**")

st.subheader("1️⃣ 지사별 계약건수 및 총금액")
if '관리지사명' in filtered.columns:
    g1 = filtered.groupby('관리지사명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g1['총금액'] = g1['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g1)
    chart1 = filtered.groupby('관리지사명')[fee_cols[0]].sum().reset_index()
    fig1 = px.bar(chart1, x='관리지사명', y=fee_cols[0], title="지사별 총금액", text=fee_cols[0])
    fig1.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig1, use_container_width=True)

st.subheader("2️⃣ 본부별 계약건수 및 총금액")
if '관리본부명' in filtered.columns:
    g2 = filtered.groupby('관리본부명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g2['총금액'] = g2['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g2)
    chart2 = filtered.groupby('관리본부명')[fee_cols[0]].sum().reset_index()
    fig2 = px.bar(chart2, x='관리본부명', y=fee_cols[0], title="본부별 총금액", text=fee_cols[0])
    fig2.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig2, use_container_width=True)

st.subheader("3️⃣ 영업자별 Top 10 실적")
if '영업자명' in filtered.columns:
    g3 = filtered.groupby('영업자명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g3 = g3.sort_values(by='총금액', ascending=False).head(10)
    g3['총금액'] = g3['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g3)
    chart3 = filtered.groupby('영업자명')[fee_cols[0]].sum().reset_index().sort_values(by=fee_cols[0], ascending=False).head(10)
    fig3 = px.bar(chart3, x='영업자명', y=fee_cols[0], title="Top 10 영업자 총금액", text=fee_cols[0])
    fig3.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig3, use_container_width=True)

st.subheader("4️⃣ 상호 키워드 워드클라우드")
if '상호' in filtered.columns:
    text = ' '.join(filtered['상호'].dropna().astype(str))
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)
'''
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

In [ ]:
from pyngrok import conf, ngrok
import os
import time

conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()

os.system('streamlit run app.py &>/content/log.txt &

SyntaxError: unterminated string literal (detected at line 8) (<ipython-input-15-de24c06e22dc>, line 8)

In [ ]:
# 쥬피터노트북용


# ✅ 1단계: 필수 패키지 설치
!pip install streamlit pyngrok plotly wordcloud matplotlib --quiet

# ✅ 2단계: 앱 코드 저장
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 데이터 전문가 분석 대시보드")

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})
df['관리지사명'] = df['관리지사명'].replace({'수성지사': '동대구지사', '춘천고객지원팀': '원주지사'})

fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

with st.sidebar:
    st.header("🔎 조건 검색")
    col1, col2 = st.columns(2)
    with col1:
        본부선택 = st.multiselect("관리본부명", options=sorted(df['관리본부명'].dropna().unique()), default=[])
    with col2:
        지사선택 = st.multiselect("관리지사명", options=sorted(df['관리지사명'].dropna().unique()), default=[])

    금액조건 = st.number_input("월정료 이상 금액", min_value=0, value=0)
    키워드 = st.text_input("계약번호 or 상호 검색", "")

filtered = df.copy()
if 본부선택:
    filtered = filtered[filtered['관리본부명'].isin(본부선택)]
if 지사선택:
    filtered = filtered[filtered['관리지사명'].isin(지사선택)]
if 금액조건 > 0 and fee_cols:
    filtered = filtered[filtered[fee_cols[0]] >= 금액조건]
if 키워드.strip():
    filtered = filtered[
        filtered['계약번호'].astype(str).str.contains(키워드, case=False) |
        filtered['상호'].astype(str).str.contains(키워드, case=False)
    ]

st.markdown(f"**데이터 건수: {len(filtered):,}건**")

st.subheader("1️⃣ 지사별 계약건수 및 총금액")
if '관리지사명' in filtered.columns:
    g1 = filtered.groupby('관리지사명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g1['총금액'] = g1['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g1)
    chart1 = filtered.groupby('관리지사명')[fee_cols[0]].sum().reset_index()
    fig1 = px.bar(chart1, x='관리지사명', y=fee_cols[0], title="지사별 총금액", text=fee_cols[0])
    fig1.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig1, use_container_width=True)

st.subheader("2️⃣ 본부별 계약건수 및 총금액")
if '관리본부명' in filtered.columns:
    g2 = filtered.groupby('관리본부명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g2['총금액'] = g2['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g2)
    chart2 = filtered.groupby('관리본부명')[fee_cols[0]].sum().reset_index()
    fig2 = px.bar(chart2, x='관리본부명', y=fee_cols[0], title="본부별 총금액", text=fee_cols[0])
    fig2.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig2, use_container_width=True)

st.subheader("3️⃣ 영업자별 Top 10 실적")
if '영업자명' in filtered.columns:
    g3 = filtered.groupby('영업자명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g3 = g3.sort_values(by='총금액', ascending=False).head(10)
    g3['총금액'] = g3['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g3)
    chart3 = filtered.groupby('영업자명')[fee_cols[0]].sum().reset_index().sort_values(by=fee_cols[0], ascending=False).head(10)
    fig3 = px.bar(chart3, x='영업자명', y=fee_cols[0], title="Top 10 영업자 총금액", text=fee_cols[0])
    fig3.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig3, use_container_width=True)

st.subheader("4️⃣ 상호 키워드 워드클라우드")
if '상호' in filtered.columns:
    text = ' '.join(filtered['상호'].dropna().astype(str))
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)

# ✅ 3단계: ngrok 실행 및 접속 링크 생성
from pyngrok import conf, ngrok
import time, os

conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()
os.system("streamlit run app.py &>/content/log.txt &")
time.sleep(5)
public_url = ngrok.connect(8501)
print(f"✅ 실행 완료: {public_url}")

UsageError: Line magic function `%%writefile` not found.


In [23]:
# ✅ 한글 폰트 설치 (나눔고딕)
!apt-get -qq install -y fonts-nanum

# ✅ 설치된 폰트 경로 확인
import os
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
print("✅ 폰트 설치 완료:", os.path.exists(font_path))

Selecting previously unselected package fonts-nanum.
(Reading database ... 126332 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
✅ 폰트 설치 완료: True


In [25]:
# ✅ 1. 필수 패키지 설치
!pip install streamlit pyngrok plotly wordcloud matplotlib --quiet
!apt-get update -qq && apt-get install -y fonts-nanum --quiet

# ✅ 2. 앱 코드 저장
app_code = """
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 데이터 전문가 분석 대시보드")

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})
df['관리지사명'] = df['관리지사명'].replace({'수성지사': '동대구지사', '춘천고객지원팀': '원주지사'})

fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

키워드 = st.text_input("계약번호 / 상호 / 설치주소 검색", "")
if 키워드.strip():
    filtered = filtered[
        filtered['계약번호'].astype(str).str.contains(키워드, case=False) |
        filtered['상호'].astype(str).str.contains(키워드, case=False) |
        filtered['설치주소'].astype(str).str.contains(키워드, case=False)
    ]

with st.sidebar:
    st.header("🔎 조건 검색")
    col1, col2 = st.columns(2)
    with col1:
        본부선택 = st.multiselect("관리본부명", options=sorted(df['관리본부명'].dropna().unique()), default=[])
    with col2:
        지사선택 = st.multiselect("관리지사명", options=sorted(df['관리지사명'].dropna().unique()), default=[])

    금액조건 = st.number_input("월정료 이상 금액", min_value=0, value=0)
    키워드 = st.text_input("계약번호 or 상호 검색", "")

filtered = df.copy()
if 본부선택:
    filtered = filtered[filtered['관리본부명'].isin(본부선택)]
if 지사선택:
    filtered = filtered[filtered['관리지사명'].isin(지사선택)]
if 금액조건 > 0 and fee_cols:
    filtered = filtered[filtered[fee_cols[0]] >= 금액조건]
if 키워드.strip():
    filtered = filtered[
        filtered['계약번호'].astype(str).str.contains(키워드, case=False) |
        filtered['상호'].astype(str).str.contains(키워드, case=False)
    ]

st.markdown(f"**데이터 건수: {len(filtered):,}건**")

st.subheader("1️⃣ 지사별 계약건수 및 총금액")
if '관리지사명' in filtered.columns:
    g1 = filtered.groupby('관리지사명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g1['총금액'] = g1['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g1)
    chart1 = filtered.groupby('관리지사명')[fee_cols[0]].sum().reset_index()
    fig1 = px.bar(chart1, x='관리지사명', y=fee_cols[0], title="지사별 총금액", text=fee_cols[0])
    fig1.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig1, use_container_width=True)

st.subheader("2️⃣ 본부별 계약건수 및 총금액")
if '관리본부명' in filtered.columns:
    g2 = filtered.groupby('관리본부명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g2['총금액'] = g2['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g2)
    chart2 = filtered.groupby('관리본부명')[fee_cols[0]].sum().reset_index()
    fig2 = px.bar(chart2, x='관리본부명', y=fee_cols[0], title="본부별 총금액", text=fee_cols[0])
    fig2.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig2, use_container_width=True)

st.subheader("3️⃣ 영업자별 Top 10 실적")
if '영업자명' in filtered.columns:
    g3 = filtered.groupby('영업자명').agg(계약건수=('계약번호', 'nunique'), 총금액=(fee_cols[0], 'sum')).reset_index()
    g3 = g3.sort_values(by='총금액', ascending=False).head(10)
    g3['총금액'] = g3['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g3)
    chart3 = filtered.groupby('영업자명')[fee_cols[0]].sum().reset_index().sort_values(by=fee_cols[0], ascending=False).head(10)
    fig3 = px.bar(chart3, x='영업자명', y=fee_cols[0], title="Top 10 영업자 총금액", text=fee_cols[0])
    fig3.update_traces(texttemplate='%{text:,} 원', textposition='outside')
    st.plotly_chart(fig3, use_container_width=True)

st.subheader("4️⃣ 상호 키워드 워드클라우드")
if '상호' in filtered.columns:
    text = ' '.join(filtered['상호'].dropna().astype(str))
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)
"""

# 저장
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

# ✅ 3. ngrok 실행
from pyngrok import conf, ngrok
import os, time

conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()
os.system("streamlit run app.py &>/content/log.txt &")
time.sleep(5)
public_url = ngrok.connect(8501)
print("✅ 접속 링크:", public_url)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists...
Building dependency tree...
Reading state information...
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
✅ 접속 링크: NgrokTunnel: "https://82ac-34-125-201-77.ngrok-free.app" -> "http://localhost:8501"


In [26]:
# ✅ Streamlit 앱 코드 저장 (설치주소, 상호, 계약번호 검색 포함, 예외처리 완비)
app_code = """
import streamlit as st
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📊 계약 데이터 전문가 분석 대시보드")

file_path = "/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})
df['관리지사명'] = df['관리지사명'].replace({'수성지사': '동대구지사', '춘천고객지원팀': '원주지사'})

fee_cols = [col for col in df.columns if '월정료' in col or '금액' in col or '정산' in col]
for col in fee_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

filtered = df.copy()

with st.sidebar:
    st.header("🔎 조건 검색")
    col1, col2 = st.columns(2)
    with col1:
        본부 = st.multiselect("관리본부명", sorted(df['관리본부명'].unique()))
    with col2:
        지사 = st.multiselect("관리지사명", sorted(df['관리지사명'].unique()))

    금액 = st.number_input("월정료 이상", min_value=0, value=0)
    키워드 = st.text_input("계약번호 / 상호 / 설치주소 검색")

if 본부:
    filtered = filtered[filtered['관리본부명'].isin(본부)]
if 지사:
    filtered = filtered[filtered['관리지사명'].isin(지사)]
if 금액 > 0 and fee_cols:
    filtered = filtered[filtered[fee_cols[0]] >= 금액]
if 키워드.strip():
    filtered = filtered[
        filtered['계약번호'].astype(str).str.contains(키워드, na=False) |
        filtered['상호'].astype(str).str.contains(키워드, na=False) |
        filtered['설치주소'].astype(str).str.contains(키워드, na=False)
    ]

st.markdown(f"**총 계약 건수: {len(filtered):,} 건**")
if not filtered.empty:
    st.dataframe(filtered[['관리본부명', '관리지사명', '계약번호', '상호', '설치주소']])
else:
    st.warning("조건에 맞는 계약이 없습니다.")

st.subheader("1️⃣ 지사별 계약")
if not filtered.empty:
    g1 = filtered.groupby('관리지사명').agg(계약수=('계약번호','nunique'), 총금액=(fee_cols[0],'sum')).reset_index()
    g1['총금액'] = g1['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g1)

st.subheader("2️⃣ 본부별 계약")
if not filtered.empty:
    g2 = filtered.groupby('관리본부명').agg(계약수=('계약번호','nunique'), 총금액=(fee_cols[0],'sum')).reset_index()
    g2['총금액'] = g2['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g2)

st.subheader("3️⃣ 영업자 Top 10")
if '영업자명' in filtered.columns and not filtered.empty:
    g3 = filtered.groupby('영업자명').agg(계약수=('계약번호','nunique'), 총금액=(fee_cols[0],'sum')).reset_index()
    g3 = g3.sort_values(by='총금액', ascending=False).head(10)
    g3['총금액'] = g3['총금액'].apply(lambda x: f"{int(x):,} 원")
    st.dataframe(g3)

st.subheader("4️⃣ 상호 워드클라우드")
if '상호' in filtered.columns and not filtered['상호'].dropna().empty:
    text = ' '.join(filtered['상호'].dropna())
    wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white', width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)
else:
    st.info("❗ 워드클라우드 생성을 위한 상호 데이터가 없습니다.")
"""

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)
print("✅ 앱 코드 저장 완료: app.py")

✅ 앱 코드 저장 완료: app.py


In [27]:
from pyngrok import conf, ngrok
import os, time

conf.get_default().auth_token = "2qPM1tLfYt5nG3zAvMvQH2AF4eg_2o2L21cnrLh5J7RgMCVuf"
ngrok.kill()
os.system("streamlit run app.py &>/content/log.txt &")
time.sleep(5)
public_url = ngrok.connect(8501)
print("✅ 접속 링크:", public_url)

✅ 접속 링크: NgrokTunnel: "https://34cf-34-125-201-77.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import pandas as pd
import requests
import time
import folium
import os

# ✅ 카카오 API 키
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# ✅ 주소 → 위도/경도, 시군구, 읍면동 추출 함수
def get_kakao_info(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    try:
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            docs = res.json().get('documents')
            if docs:
                doc = docs[0]
                lat = doc['y']
                lon = doc['x']
                region_2depth = doc['address']['region_2depth_name']
                region_3depth = doc['address']['region_3depth_name']
                url = f"https://map.kakao.com/link/map/{address},{lat},{lon}"
                return lat, lon, region_2depth, region_3depth, url
    except:
        pass
    return None, None, None, None, None

# ✅ CSV 파일 로드
file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df = df.dropna(subset=['설치주소'])

# ✅ 관리본부명 보정
df['관리본부명'] = df['관리본부명'].replace({
    '강원본부': '강북/강원본부',
    '서부본부': '강남/서부본부'
})

# ✅ 주소 기반 정보 추출
latitudes, longitudes, sigungus, eupmyeondongs, urls = [], [], [], [], []

for i, addr in enumerate(df['설치주소']):
    lat, lon, si, eup, url = get_kakao_info(addr)
    latitudes.append(lat)
    longitudes.append(lon)
    sigungus.append(si)
    eupmyeondongs.append(eup)
    urls.append(url)

    if i % 50 == 0:
        print(f"▶️ {i}건 처리 중...")
    time.sleep(0.2)

# ✅ 컬럼 추가 및 결측 제거
df['위도'] = latitudes
df['경도'] = longitudes
df['시군구'] = sigungus
df['읍면동'] = eupmyeondongs
df['지도링크'] = urls
df = df.dropna(subset=['위도', '경도'])

# ✅ 지도 시각화
m = folium.Map(location=[float(df['위도'].mean()), float(df['경도'].mean())], zoom_start=10)

for _, row in df.iterrows():
    popup_html = f"""
    <b>관리본부:</b> {row.get('관리본부명', '')}<br>
    <b>관리지사:</b> {row.get('관리지사명', '')}<br>
    <b>계약번호:</b> {row.get('계약번호', '')}<br>
    <b>상호:</b> {row.get('상호', '')}<br>
    <b>영업구역:</b> {row.get('영업구역정보', '')}<br>
    <b>주소:</b> {row.get('설치주소', '')}<br>
    <b>시군구:</b> {row.get('시군구', '')} / <b>읍면동:</b> {row.get('읍면동', '')}<br>
    <a href="{row.get('지도링크', '')}" target="_blank">카카오맵에서 보기</a>
    """
    folium.CircleMarker(
        location=[float(row['위도']), float(row['경도'])],
        radius=5,
        popup=folium.Popup(popup_html, max_width=400),
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7
    ).add_to(m)

# ✅ 저장
output_path = '/content/drive/MyDrive/지도시각화/3월_설치주소_지도_위경도_포함(25.4.22).html'
m.save(output_path)
print(f"✅ 지도 저장 완료: {output_path}")

▶️ 0건 처리 중...


KeyboardInterrupt: 